# Extract YRBS Data

Youth Risk Behavior Survey (YRBS) data has been converted from fixed-width-format, to comma separated values.  The dataset was split into two by state name, A-M and N-Z. In this notebook, we will merge the two together and perform some basic cleanup and initial exploration.

This notebook saves results in local SQL database table `STATE`.


In [1]:
import sys
import time
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import inspect, create_engine, Table, Column, Integer, String

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

start_time = time.time()

In [2]:
!nvidia-smi

Sun Oct 16 10:34:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.96       Driver Version: 512.96       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  Off |
|  0%   37C    P8    19W / 450W |    415MiB / 24564MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Set the file paths and read data

TODO:  Review other csv files and find another cloud storage option.

Download the converted CSV files from here: https://drive.google.com/file/d/11KfEMD5GrSDqIwB6jURa45W0fX2yezXF/view?usp=sharing

Then extract the contents of the zip file into the `data` directory. You should have four csv files:

        <project root>
        .
        ├── LICENSE
        ├── README.md
        ├── data
        │   └── README.md
        ├── notebooks
        │   ├── 01_Extract.ipynb
        │   ├── 02_Transform.ipynb
        │   ├── 02_Transform_Chicago.ipynb
        │   └── 02_Transform_NYC.ipynb
        ├── origin_data
        │   ├── README.md
        │   ├── SADC_District_Q.csv
        │   ├── SADC_National_Q.csv
        │   ├── SADC_State_a_m_Q.csv
        │   └── SADC_State_n_z_Q.csv
        └── src
            ├── connection.r
            ├── dictionaries.py
            ├── labels.py
            ├── test.py
            └── utils.py
---


In [3]:
state_a_m_datafile = Path("../origin_data/SADC_State_a_m_Q.csv")
state_n_z_datafile = Path("../origin_data/SADC_State_n_z_Q.csv")
district_datafile = Path("../origin_data/SADC_District_Q.csv")
national_datafile = Path("../origin_data/SADC_National_Q.csv")
state_a_m_df = pd.read_csv(state_a_m_datafile)
state_n_z_df = pd.read_csv(state_n_z_datafile)
district_df = pd.read_csv(district_datafile)
national_df = pd.read_csv(national_datafile)
state_df = pd.concat([state_a_m_df, state_n_z_df])


## Data Cleanup

In [4]:
# # convert `year` to int
# district_df["year"] = district_df["year"].astype('int')
# state_df["year"] = state_df["year"].astype('int')
# national_df["year"] = national_df["year"].astype('int')
# state_df.iloc[:, 4:14].head()

## Address any NaN values



In [5]:
# Convert categorized `object` data to int and fillna() with `0`
"""NOTE:
Convert these categorical columns to int and fill null values with 0.

The data source has mixed types throughout.  Once pulled into pandas, cleanup
is straight-forward. FIll NaNs with `0`s and set to int.

"""
cols_int = ["sitetypenum", "year", "survyear", "stratum", "PSU", "record", "age", "sex", "grade", "race4", "race7", "qnobese", "qnowt", "q66", "q65", "sexid", "sexid2", "sexpart", "sexpart2", 
            "q8", "q9", "q10", "q11", "q12", "q13", "q14", "q15", "q16", "q17", "q18", "q19", "q20", "q21", "q22", "q23", "q24", "q25", "q26", "q27", "q28", "q29", 
            "q30", "q31", "q32", "q33", "q34", "q35", "q36", "q37", "q38", "q39", "q40", "q41", "q42", "q43", "q44", "q45", "q46", "q47", "q48", "q49", 
            "q50", "q51", "q52", "q53", "q54", "q55", "q56", "q57", "q58", "q59", "q60", "q61", "q62", "q63", "q64", "q67", "q68", "q69", 
            "q70", "q71", "q72", "q73", "q74", "q75", "q76", "q77", "q78", "q79", "q80", "q81", "q82", "q83", "q84", "q85", "q86", "q87", "q88", "q89", 
            "qbikehelmet", "qdrivemarijuana", "qcelldriving", "qpropertydamage", "qbullyweight", "qbullygender", "qbullygay", "qchokeself", "qcigschool", 
            "qchewtobschool", "qalcoholschool", "qtypealcohol2", "qhowmarijuana", "qmarijuanaschool", "qcurrentopioid", "qcurrentcocaine", "qcurrentheroin", "qcurrentmeth", 
            "qhallucdrug", "qprescription30d", "qgenderexp", "qtaughtHIV", "qtaughtsexed", "qtaughtstd", "qtaughtcondom", "qtaughtbc", "qdietpop", "qcoffeetea", "qsportsdrink", 
            "qenergydrink", "qsugardrink", "qwater", "qfastfood", "qfoodallergy", "qwenthungry", "qmusclestrength", "qsunscreenuse", "qindoortanning", "qsunburn", "qconcentrating", 
            "qcurrentasthma", "qwheresleep", "qspeakenglish", "qtransgender"]

for col in cols_int:
    state_df[col] = state_df[col].fillna(0)
    state_df[col] = state_df[col].astype('int')
    district_df[col] = district_df[col].fillna(0)
    district_df[col] = district_df[col].astype('int')
    national_df[col] = national_df[col].fillna(0)
    national_df[col] = national_df[col].astype('int')


In [9]:
# Move record column to the front of the dataframe
def move_column_inplace(df, col, pos):
    col = df.pop(col)
    df.insert(pos, col.name, col)

def move_to_front_inplace(df, col):
    move_column_inplace(df, col, 0)
    return df

col = "record"
district_df = move_to_front_inplace(district_df.copy(), col)
state_df = move_to_front_inplace(state_df.copy(), col)
national_df = move_to_front_inplace(national_df.copy(), col)

In [15]:
display(district_df.head(3))
display(state_df.head(3))
display(national_df.head(3))

,record,sitecode,sitename,sitetype,sitetypenum,year,survyear,weight,stratum,PSU,age,sex,grade,race4,race7,stheight,stweight,bmi,bmipct,qnobese,qnowt,q66,q65,sexid,sexid2,sexpart,sexpart2,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50,q51,q52,q53,q54,q55,q56,q57,q58,q59,q60,q61,q62,q63,q64,q67,q68,q69,q70,q71,q72,q73,q74,q75,q76,q77,q78,q79,q80,q81,q82,q83,q84,q85,q86,q87,q88,q89,qbikehelmet,qdrivemarijuana,qcelldriving,qpropertydamage,qbullyweight,qbullygender,qbullygay,qchokeself,qcigschool,qchewtobschool,qalcoholschool,qtypealcohol2,qhowmarijuana,qmarijuanaschool,qcurrentopioid,qcurrentcocaine,qcurrentheroin,qcurrentmeth,qhallucdrug,qprescription30d,qgenderexp,qtaughtHIV,qtaughtsexed,qtaughtstd,qtaughtcondom,qtaughtbc,qdietpop,qcoffeetea,qsportsdrink,qenergydrink,qsugardrink,qwater,qfastfood,qfoodallergy,qwenthungry,qmusclestrength,qsunscreenuse,qindoortanning,qsunburn,qconcentrating,qcurrentasthma,qwheresleep,qspeakenglish,qtransgender
0,1776279,NYI,"Borough of Manhattan, NY (NYI)",District,1,2007,9,3.71,49,2,5,1,2,3,4,1.73,92.99,31.07,96.60,1,2,1,3,1,1,2,2,5,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,1,1,0,0,1,0,1,1,0,0,0,0,1,0,5,0,0,0,0,5,4,1,0,0,1,1,1,1,1,1,1,2,0,0,0,0,0,0,0,4,1,1,1,0,0,0,0,7,2,0,1,7,7,1,3,0,0,0,0,2,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1776280,NYI,"Borough of Manhattan, NY (NYI)",District,1,2007,9,7.38,49,2,5,1,1,2,3,1.70,77.11,26.68,90.68,2,1,0,0,0,0,0,0,5,0,0,0,1,1,0,1,1,1,1,2,0,0,0,0,0,2,2,2,1,1,2,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1776281,NYI,"Borough of Manhattan, NY (NYI)",District,1,2007,9,3.89,49,2,5,1,4,3,4,1.60,49.90,19.49,33.66,2,2,1,1,1,1,1,1,4,0,0,0,1,1,0,1,1,1,1,2,0,0,0,0,0,1,2,2,1,1,1,0,1,1,0,0,0,0,1,0,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,3,1,2,4,0,0,0,0,3,2,0,4,4,2,1,1,0,0,0,0,2,0,0,2,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,record,sitecode,sitename,sitetype,sitetypenum,year,survyear,weight,stratum,PSU,age,sex,grade,race4,race7,stheight,stweight,bmi,bmipct,qnobese,qnowt,q66,q65,sexid,sexid2,sexpart,sexpart2,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50,q51,q52,q53,q54,q55,q56,q57,q58,q59,q60,q61,q62,q63,q64,q67,q68,q69,q70,q71,q72,q73,q74,q75,q76,q77,q78,q79,q80,q81,q82,q83,q84,q85,q86,q87,q88,q89,qbikehelmet,qdrivemarijuana,qcelldriving,qpropertydamage,qbullyweight,qbullygender,qbullygay,qchokeself,qcigschool,qchewtobschool,qalcoholschool,qtypealcohol2,qhowmarijuana,qmarijuanaschool,qcurrentopioid,qcurrentcocaine,qcurrentheroin,qcurrentmeth,qhallucdrug,qprescription30d,qgenderexp,qtaughtHIV,qtaughtsexed,qtaughtstd,qtaughtcondom,qtaughtbc,qdietpop,qcoffeetea,qsportsdrink,qenergydrink,qsugardrink,qwater,qfastfood,qfoodallergy,qwenthungry,qmusclestrength,qsunscreenuse,qindoortanning,qsunburn,qconcentrating,qcurrentasthma,qwheresleep,qspeakenglish,qtransgender
0,467788,FL,Florida (FL),State,2,2007,9,144.85,22,2,5,2,2,3,4,1.88,93.44,26.43,92.05,2,1,0,0,0,0,0,0,5,1,0,0,1,1,0,1,1,1,1,2,0,0,0,0,0,2,2,2,0,0,0,0,1,0,0,0,0,0,1,0,5,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,2,1,4,7,3,3,2,4,3,1,5,2,2,1,1,3,5,2,0,0,4,5,6,1,0,0,0,0,2,0,0,2,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,3,0,0,0
1,467789,FL,Florida (FL),State,2,2007,9,144.85,22,2,5,2,2,3,4,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,4,2,0,0,1,1,0,1,1,1,1,2,0,0,0,0,0,2,2,2,1,1,0,0,1,0,0,0,0,0,1,0,3,1,0,0,1,7,4,0,0,0,1,1,1,1,1,1,1,1,1,3,3,2,3,3,7,4,1,2,2,3,2,1,2,1,2,0,0,6,1,1,2,0,0,0,0,1,0,0,2,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,4,0,0,0
2,467790,FL,Florida (FL),State,2,2007,9,144.85,22,2,5,2,2,3,4,1.75,97.52,31.84,98.35,1,2,0,0,0,0,0,0,4,2,0,0,4,3,0,1,3,4,3,2,0,0,0,0,0,1,2,2,1,1,0,0,6,0,0,0,0,0,6,0,2,3,0,0,6,6,4,3,0,0,2,1,1,1,1,1,1,2,1,2,5,2,3,3,2,4,1,2,2,2,2,1,2,3,2,0,0,6,5,1,2,0,0,0,0,2,0,0,2,0,0,4,0,0,0,0,2,0,1,0,0,1,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0


,record,sitecode,sitename,sitetype,sitetypenum,year,survyear,weight,stratum,PSU,age,sex,grade,race4,race7,stheight,stweight,bmi,bmipct,qnobese,qnowt,q66,q65,sexid,sexid2,sexpart,sexpart2,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50,q51,q52,q53,q54,q55,q56,q57,q58,q59,q60,q61,q62,q63,q64,q67,q68,q69,q70,q71,q72,q73,q74,q75,q76,q77,q78,q79,q80,q81,q82,q83,q84,q85,q86,q87,q88,q89,qbikehelmet,qdrivemarijuana,qcelldriving,qpropertydamage,qbullyweight,qbullygender,qbullygay,qchokeself,qcigschool,qchewtobschool,qalcoholschool,qtypealcohol2,qhowmarijuana,qmarijuanaschool,qcurrentopioid,qcurrentcocaine,qcurrentheroin,qcurrentmeth,qhallucdrug,qprescription30d,qgenderexp,qtaughtHIV,qtaughtsexed,qtaughtstd,qtaughtcondom,qtaughtbc,qdietpop,qcoffeetea,qsportsdrink,qenergydrink,qsugardrink,qwater,qfastfood,qfoodallergy,qwenthungry,qmusclestrength,qsunscreenuse,qindoortanning,qsunburn,qconcentrating,qcurrentasthma,qwheresleep,qspeakenglish,qtransgender
0,721036,XX,United States (XX),National,3,2017,14,0.74,103,51190,5,2,3,2,3,1.63,84.82,31.92,98.39,1,2,4,2,4,3,2,2,5,1,2,2,1,1,1,1,1,8,5,2,1,1,1,1,2,2,2,2,1,1,1,6,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,2,1,7,3,2,3,2,4,4,4,2,1,2,2,2,3,2,4,5,4,2,7,6,1,1,2,0,1,1,2,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,7,0,1,0,1,0,1,1,1,0,0,1,0
1,721037,XX,United States (XX),National,3,2017,14,4.58,113,261631,5,2,3,3,4,1.73,74.39,24.85,86.48,2,1,1,2,1,1,2,2,5,2,2,5,1,1,1,1,1,1,1,2,1,2,2,2,2,2,2,2,1,1,1,5,2,2,1,2,5,2,2,3,6,2,1,2,6,5,5,2,1,1,1,1,1,1,1,1,1,1,1,5,3,3,3,2,4,3,3,2,4,1,2,1,3,2,1,8,7,3,2,1,2,1,2,0,1,2,5,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,7,0,2,0,7,0,1,2,2,0,0,1,0
2,721038,XX,United States (XX),National,3,2017,14,1.97,211,480850,5,2,2,1,6,1.85,83.92,24.52,84.84,2,2,1,2,1,1,2,2,5,1,2,2,1,1,1,1,1,2,1,2,1,2,2,2,2,2,2,2,1,1,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,1,1,1,1,1,1,3,3,1,1,2,1,1,1,2,2,8,7,4,3,1,2,1,2,0,1,2,4,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,3,0,2,0,6,0,1,2,2,0,0,2,0


## Save Imported Data to SQL Database Tables

In [16]:
connection_string = 'sqlite:///../data/yrbss_data.db'
engine = create_engine(connection_string)
insp = inspect(engine)

In [17]:
start_time = time.time()
state_df.to_sql('STATE', engine, index=False, if_exists='replace')
# print(insp.get_table_names())
# display(pd.read_sql_query("SELECT * FROM STATE LIMIT 3;", con=engine))
# display(pd.read_sql_query("SELECT COUNT() FROM STATE", con=engine))
current_time = time.time()
elapsed_time = current_time - start_time
print(f"STATE import took {int(elapsed_time)} seconds")

STATE import took 88 seconds


In [18]:
start_time = time.time()
district_df.to_sql('DISTRICT', engine, index=False, if_exists='replace')
# print(insp.get_table_names())
# display(pd.read_sql_query("SELECT * FROM STATE LIMIT 3;", con=engine))
# display(pd.read_sql_query("SELECT COUNT() FROM STATE", con=engine))
current_time = time.time()
elapsed_time = current_time - start_time
print(f"DISTRICT table import took {int(elapsed_time)} seconds")

DISTRICT table import took 28 seconds


In [19]:
start_time = time.time()
national_df.to_sql('NATIONAL', engine, index=False, if_exists='replace')
# display(pd.read_sql_query("SELECT * FROM STATE LIMIT 3;", con=engine))
# display(pd.read_sql_query("SELECT COUNT() FROM STATE", con=engine))
current_time = time.time()
elapsed_time = current_time - start_time
print(f"NATIONAL table import took {int(elapsed_time)} seconds")
print(insp.get_table_names())

NATIONAL table import took 11 seconds
['DISTRICT', 'NATIONAL', 'STATE']


## Read Sample Data into Pandas

In [20]:
my_subset_df = pd.read_sql_query("SELECT * FROM NATIONAL;", con=engine)
my_subset_df.shape

(217340, 151)

In [21]:
print(f"Imported from SQL Dataframe Shape: {my_subset_df.shape}\n\n")
print("Head:")
display(my_subset_df.head(3))
print("Tail:")
display(my_subset_df.tail(3))

Imported from SQL Dataframe Shape: (217340, 151)


Head:


,record,sitecode,sitename,sitetype,sitetypenum,year,survyear,weight,stratum,PSU,age,sex,grade,race4,race7,stheight,stweight,bmi,bmipct,qnobese,qnowt,q66,q65,sexid,sexid2,sexpart,sexpart2,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50,q51,q52,q53,q54,q55,q56,q57,q58,q59,q60,q61,q62,q63,q64,q67,q68,q69,q70,q71,q72,q73,q74,q75,q76,q77,q78,q79,q80,q81,q82,q83,q84,q85,q86,q87,q88,q89,qbikehelmet,qdrivemarijuana,qcelldriving,qpropertydamage,qbullyweight,qbullygender,qbullygay,qchokeself,qcigschool,qchewtobschool,qalcoholschool,qtypealcohol2,qhowmarijuana,qmarijuanaschool,qcurrentopioid,qcurrentcocaine,qcurrentheroin,qcurrentmeth,qhallucdrug,qprescription30d,qgenderexp,qtaughtHIV,qtaughtsexed,qtaughtstd,qtaughtcondom,qtaughtbc,qdietpop,qcoffeetea,qsportsdrink,qenergydrink,qsugardrink,qwater,qfastfood,qfoodallergy,qwenthungry,qmusclestrength,qsunscreenuse,qindoortanning,qsunburn,qconcentrating,qcurrentasthma,qwheresleep,qspeakenglish,qtransgender
0,721036,XX,United States (XX),National,3,2017,14,0.74,103,51190,5,2,3,2,3,1.63,84.82,31.92,98.39,1,2,4,2,4,3,2,2,5,1,2,2,1,1,1,1,1,8,5,2,1,1,1,1,2,2,2,2,1,1,1,6,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,2,1,7,3,2,3,2,4,4,4,2,1,2,2,2,3,2,4,5,4,2,7,6,1,1,2,0,1,1,2,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,7,0,1,0,1,0,1,1,1,0,0,1,0
1,721037,XX,United States (XX),National,3,2017,14,4.58,113,261631,5,2,3,3,4,1.73,74.39,24.85,86.48,2,1,1,2,1,1,2,2,5,2,2,5,1,1,1,1,1,1,1,2,1,2,2,2,2,2,2,2,1,1,1,5,2,2,1,2,5,2,2,3,6,2,1,2,6,5,5,2,1,1,1,1,1,1,1,1,1,1,1,5,3,3,3,2,4,3,3,2,4,1,2,1,3,2,1,8,7,3,2,1,2,1,2,0,1,2,5,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,7,0,2,0,7,0,1,2,2,0,0,1,0
2,721038,XX,United States (XX),National,3,2017,14,1.97,211,480850,5,2,2,1,6,1.85,83.92,24.52,84.84,2,2,1,2,1,1,2,2,5,1,2,2,1,1,1,1,1,2,1,2,1,2,2,2,2,2,2,2,1,1,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,1,1,1,1,1,1,3,3,1,1,2,1,1,1,2,2,8,7,4,3,1,2,1,2,0,1,2,4,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,3,0,2,0,6,0,1,2,2,0,0,2,0


Tail:


,record,sitecode,sitename,sitetype,sitetypenum,year,survyear,weight,stratum,PSU,age,sex,grade,race4,race7,stheight,stweight,bmi,bmipct,qnobese,qnowt,q66,q65,sexid,sexid2,sexpart,sexpart2,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45,q46,q47,q48,q49,q50,q51,q52,q53,q54,q55,q56,q57,q58,q59,q60,q61,q62,q63,q64,q67,q68,q69,q70,q71,q72,q73,q74,q75,q76,q77,q78,q79,q80,q81,q82,q83,q84,q85,q86,q87,q88,q89,qbikehelmet,qdrivemarijuana,qcelldriving,qpropertydamage,qbullyweight,qbullygender,qbullygay,qchokeself,qcigschool,qchewtobschool,qalcoholschool,qtypealcohol2,qhowmarijuana,qmarijuanaschool,qcurrentopioid,qcurrentcocaine,qcurrentheroin,qcurrentmeth,qhallucdrug,qprescription30d,qgenderexp,qtaughtHIV,qtaughtsexed,qtaughtstd,qtaughtcondom,qtaughtbc,qdietpop,qcoffeetea,qsportsdrink,qenergydrink,qsugardrink,qwater,qfastfood,qfoodallergy,qwenthungry,qmusclestrength,qsunscreenuse,qindoortanning,qsunburn,qconcentrating,qcurrentasthma,qwheresleep,qspeakenglish,qtransgender
217337,2258521,XX,United States (XX),National,3,2019,15,1.10,201,330220,7,1,4,1,6,1.63,68.04,25.60,83.90,2,2,1,3,1,1,2,2,5,0,2,0,0,1,0,1,1,1,1,2,0,0,4,2,2,2,2,2,1,0,1,0,7,4,1,5,4,1,1,0,5,2,0,3,8,3,4,1,0,1,0,1,0,0,0,0,0,2,1,3,7,2,3,2,4,0,0,1,2,0,0,0,0,0,2,0,3,4,2,1,0,0,2,0,0,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
217338,2258522,XX,United States (XX),National,3,2019,15,1.77,112,343678,7,1,3,0,0,1.68,55.79,19.76,27.55,2,2,0,0,0,0,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,0,1,2,0,2,2,2,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,1,1,1,1,1,1,0,1,0,0,0,1,2,2,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
217339,2258523,XX,United States (XX),National,3,2019,15,4.74,112,343678,7,1,4,2,3,1.65,52.16,19.15,19.47,2,2,1,0,1,1,0,0,0,1,1,1,0,1,1,1,1,0,1,0,0,0,1,2,2,1,2,2,0,0,0,1,1,0,2,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,2,2,1,1,1,1,1,1,2,0,2,5,2,1,1,4,1,0,8,6,1,7,3,0,1,2,0,1,2,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
